<a href="https://www.kaggle.com/code/srsses/tabular-playground-jan-2022-using-xgboost?scriptVersionId=156349623" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
s_t = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv")

In [3]:
s_t.head()

,row_id,date,country,store,product,num_sold
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911


In [4]:
s_t.shape

(26298, 6)

In [5]:
s_t.isnull().sum()

row_id      0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

In [6]:
s_t.nunique()

row_id      26298
date         1461
country         3
store           2
product         3
num_sold     1377
dtype: int64

**Creating month,day,weekday from the date column**

In [7]:
from datetime import date

In [8]:
s_t['date'] = pd.to_datetime(s_t.date)

In [9]:
s_t['day'] = s_t.date.apply(lambda x:x.day)
s_t['month'] = s_t.date.apply(lambda x: x.month)
s_t['weekday'] = s_t.date.apply(lambda x:x.dayofweek)

In [10]:
s_t.head()

,row_id,date,country,store,product,num_sold,day,month,weekday
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329,1,1,3
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520,1,1,3
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146,1,1,3
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572,1,1,3
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911,1,1,3


**lets drop the row_id and date column as they are not usefull**

In [11]:
s_t.drop(['row_id','date'],axis=1,inplace = True)

In [12]:
for i in list(s_t.columns):
    print(s_t[i].unique())

['Finland' 'Norway' 'Sweden']
['KaggleMart' 'KaggleRama']
['Kaggle Mug' 'Kaggle Hat' 'Kaggle Sticker']
[ 329  520  146 ... 1231 1124 2128]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
[ 1  2  3  4  5  6  7  8  9 10 11 12]
[3 4 5 6 0 1 2]


In [13]:
def product(c):
    l = c.split(' ')
    return l[1]

In [14]:
s_t['product'] = s_t['product'].apply(product)

In [15]:
s_t.head(2)

,country,store,product,num_sold,day,month,weekday
0,Finland,KaggleMart,Mug,329,1,1,3
1,Finland,KaggleMart,Hat,520,1,1,3


**imported the necessary encoders but they are not required**

In [16]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,LabelBinarizer

In [17]:
ohc = OneHotEncoder()
le = LabelEncoder()
lb = LabelBinarizer()
col = ['country','store','product']
s_t[col].head().T
s_t = pd.get_dummies(s_t, columns=col, drop_first=False)
# for i in col:
#     if s_t[i].nunique() == 2:
#         s_t[i] = lb.fit_transform(s_t[i]).astype(int)
#     else:
#         s_t[i] = le.fit_transform(s_t[i]).astype(int)

In [18]:
s_t.head()

,num_sold,day,month,weekday,country_Finland,country_Norway,country_Sweden,store_KaggleMart,store_KaggleRama,product_Hat,product_Mug,product_Sticker
0,329,1,1,3,1,0,0,1,0,0,1,0
1,520,1,1,3,1,0,0,1,0,1,0,0
2,146,1,1,3,1,0,0,1,0,0,0,1
3,572,1,1,3,1,0,0,0,1,0,1,0
4,911,1,1,3,1,0,0,0,1,1,0,0


**Process the test data in same way**

In [19]:
tst = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
tstc = tst.copy()
tst['date'] = pd.to_datetime(tst.date)
tst['day'] = tst.date.apply(lambda x:x.day)
tst['month'] = tst.date.apply(lambda x: x.month)
tst['weekday'] = tst.date.apply(lambda x:x.dayofweek)
tst.drop(['row_id','date'],axis=1,inplace = True)
tst['product'] = tst['product'].apply(product)
col = ['country','store','product']
tst = pd.get_dummies(tst, columns=col, drop_first=False)
tst.head(2)

,day,month,weekday,country_Finland,country_Norway,country_Sweden,store_KaggleMart,store_KaggleRama,product_Hat,product_Mug,product_Sticker
0,1,1,1,1,0,0,1,0,0,1,0
1,1,1,1,1,0,0,1,0,1,0,0


**create the train and target data to train, use train_test_split to test different regression techniques and import necessary libraries. I have scaled the data using minmax scaler mostly**

In [20]:
x_d = s_t.drop('num_sold',axis = 1)
y_d = s_t['num_sold']

In [21]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
lr = LinearRegression()
rr = Ridge(max_iter = 10000,alpha = 10,fit_intercept = True,random_state = 34)
ls = Lasso(max_iter = 1000000,alpha = 0.0005,fit_intercept = True,random_state = 34)

In [22]:
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PolynomialFeatures
from sklearn.pipeline import Pipeline
s = StandardScaler()
m = MinMaxScaler()
kf = KFold(n_splits = 8,random_state = 32,shuffle = True)
pf = PolynomialFeatures()
from scipy.stats import boxcox
from scipy.special import inv_boxcox

In [23]:
from sklearn.metrics import r2_score,accuracy_score,mean_squared_error

In [24]:
x_train,x_test,y_train,y_test = train_test_split(x_d,y_d,test_size = 0.2)

**lets test with all the regression techniques and i will use m1 which is the scaled version of x_d and y which is transformed using boxcox method. M1 and M2 and Y are defined after two steps**

In [25]:
mm = m.fit_transform(x_train)
mm1 = m.transform(x_test)
tr = boxcox(y_train)
y = tr[0]
lam = tr[1]

In [26]:
ls.fit(mm,y)

Lasso(alpha=0.0005, max_iter=1000000, random_state=34)

In [27]:
y_pred = ls.predict(mm1)
f_y_pred = inv_boxcox(y_pred,lam)

In [28]:
r2_score(y_test,f_y_pred)

0.8412396101527488

In [29]:
m1 = m.fit_transform(x_train)
m2 = m.transform(x_test)
rr.fit(m1,y)
r_pred = rr.predict(m2)

In [30]:
new_y = inv_boxcox(r_pred,lam)

In [31]:
r2_score(y_test,new_y)

0.8422130055397145

In [32]:
from sklearn.neighbors import KNeighborsRegressor
kn= KNeighborsRegressor(n_neighbors = 5)
kn.fit(m1,y)
k_pred = kn.predict(m2)

In [33]:
new_k = inv_boxcox(k_pred,lam)
r2_score(y_test,new_k)

0.9147045825241363

In [34]:
from sklearn.svm import SVR

In [35]:
sv_l = SVR(kernel = 'rbf',C = 12.0,gamma = 'scale',max_iter = 100000000)

In [36]:
sv_l.fit(m1,y)
sv_pred = sv_l.predict(m2)

In [37]:
f_sv = inv_boxcox(sv_pred,lam)
r2_score(y_test,f_sv)

0.9282539117304953

In [38]:
# from sklearn.tree import DecisionTreeRegressor

In [39]:
# dc = DecisionTreeRegressor(criterion = 'mse',max_depth = 10,max_features = 'auto',random_state = 34)

In [40]:
# dc.fit(m1,y)

In [41]:
# dc_r = dc.predict(m2)
# f_dc = inv_boxcox(dc_r,lam)
# r2_score(y_test,f_dc)

In [42]:
# from sklearn.ensemble import BaggingRegressor

In [43]:
# bg = BaggingRegressor(n_estimators = 500,max_features = 11)

In [44]:
# bg.fit(m1,y)

In [45]:
# bg_p = bg.predict(m2)
# f_bg = inv_boxcox(bg_p,lam)
# r2_score(y_test,f_bg)

In [46]:
# from sklearn.ensemble import RandomForestRegressor

In [47]:
# rf = RandomForestRegressor(n_estimators = 500,criterion = 'mse',max_depth = 10,max_features = 'auto',random_state = 34)
# rf.fit(m1,y)

In [48]:
# rfs = rf.predict(m2)
# rf_s = inv_boxcox(rfs,lam)
# r2_score(y_test,rf_s)

In [49]:
# from sklearn.ensemble import GradientBoostingRegressor

In [50]:
# gr = GradientBoostingRegressor(learning_rate=0.1,n_estimators = 1000,criterion = 'mse',max_features = 'sqrt',max_depth = 15,random_state = 34,)

In [51]:
# gr.fit(m1,y)

In [52]:
# y_gr = gr.predict(m2)
# fgr = inv_boxcox(y_gr,lam)
# r2_score(y_test,fgr)

In [53]:
# from sklearn.ensemble import AdaBoostRegressor

In [54]:
# abc = AdaBoostRegressor(n_estimators = 900,learning_rate = 0.2,loss = 'linear')

In [55]:
# abc.fit(m1,y)

In [56]:
# ab = abc.predict(m2)
# fab = inv_boxcox(ab,lam)
# r2_score(y_test,fab)

**We see ;the RandomForestRegressor result looks more promising and we will use that to train the whole training dataset and predict over the test dataset**

In [57]:
# m3 = m.fit_transform(x_d)
# m4 = m.transform(tst)


# rf_y = boxcox(y_d)
# rfy = rf_y[0]
# lam1 = rf_y[1]


# rf.fit(m3,rfy)
# prf = rf.predict(m4)


# rfn = inv_boxcox(prf,lam1)

***The score is now 6.00 and rank is 327***

In [58]:
# xgb_params = {
#         'tree_method': 'hist',
#         'grow_policy' : 'lossguide',
#         'learning_rate': 0.03399878704233446,
#         'max_depth': 5,
#         'reg_alpha': 0.7814373604498039,
#         'reg_lambda': 0.00018093104956619317,
#         'max_delta_step': 2,
#         'min_child_weight': 14,
#         'colsample_bytree': 0.6489299778623602,
#         'subsample': 0.6033298718112065,
#        lea 'max_ves': 187,  
#         }

In [59]:
from xgboost import XGBRegressor
xg = XGBRegressor(n_estimators = 10000,grow_policy ='lossguide',learning_rate = 0.06399878704233446, max_depth = 18 ,reg_alpha =  0.2814373604498039,reg_lambda= 0.80018093104956619317,colsample_bytree= 0.9589299778623602,subsample=0.9833298718112065,max_leaves= 187,tree_method= 'hist',max_delta_step= 3,booster = 'gbtree')
xg.fit(m1,y)
xgb = xg.predict(m2)
fx = inv_boxcox(xgb,lam)
r2_score(y_test,fx)

0.9871365296281819

In [60]:
m5 = m.fit_transform(x_d)
m6 = m.transform(tst)

xg_y = boxcox(y_d)
xgy = xg_y[0]
lam2 = xg_y[1]

xg.fit(m5,xgy)
prx = xg.predict(m6)

xgt = inv_boxcox(prx,lam2)


final = pd.DataFrame(xgt,columns = ['num_sold'])
final['row_id'] = tstc.row_id
final.to_csv('submission.csv',index=False)

***XGboost regressor score is  5.85 now and i will try to experiment more to improve it.***